In [72]:
# Importación de librerías
import pandas as pd  # Para manipulación y análisis de datos
from sklearn.model_selection import train_test_split  # Para dividir el conjunto de datos en entrenamiento y prueba
from sklearn.preprocessing import StandardScaler  # Para estandarizar características
from tensorflow.keras.models import Sequential  # Para inicializar modelos secuenciales
from tensorflow.keras.layers import Dense, Input  # Para definir capas densamente conectadas en la red neuronal
import joblib  # Para guardar y cargar modelos entrenados
import numpy as np  # Para cálculos numéricos
from sklearn.metrics import mean_squared_error, r2_score  # Para calcular métricas de rendimiento del modelo

In [83]:
# Cargar datos
df = pd.read_csv("juadores_fantasy.csv.csv")

#Impresion de las columnas existentes y la columna posicion 
print(df.columns)
print("\n")
print(df["posicion"])

Index(['Nombre', 'Precio', 'Media', 'Total_puntos', 'posicion', 'equipo'], dtype='object')


0      mediocentro
1      mediocentro
2        delantero
3        delantero
4        delantero
          ...     
509        portero
510        portero
511    mediocentro
512    mediocentro
513        defensa
Name: posicion, Length: 514, dtype: object


In [84]:
# Dividir los datos en características (X) y el objetivo (y)
X_Todo = df[['Precio', 'Media', 'Total_puntos']]
y_puntos = df['Total_puntos']
y_precio = df['Precio']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_puntos_train, y_puntos_test = train_test_split(X_Todo, y_puntos, test_size=0.2, random_state=42)
X_train, X_test, y_precio_train, y_precio_test = train_test_split(X_Todo, y_precio, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [85]:
# Crear el modelo para predecir puntos
model_puntos = Sequential([
    Input(shape=(X_train.shape[1],)),  
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

# Configuración del modelo para que utilice el algoritmo de optimización 
model_puntos.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo para predecir puntos
model_puntos.fit(X_train_scaled, y_puntos_train, epochs=50, batch_size=32, verbose=0)


In [86]:
# Crear el modelo para predecir precio
model_precio = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

# Configuración del modelo para que utilice el algoritmo de optimización 
model_precio.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo para predecir precio
model_precio.fit(X_train_scaled, y_precio_train, epochs=50, batch_size=32, verbose=0)


In [87]:
# Hacer predicciones para puntos y precio
puntos_predictions = model_puntos.predict(X_test_scaled).flatten()
precio_predictions = model_precio.predict(X_test_scaled).flatten()

# Crear un DataFrame con las predicciones y el índice original
predictions_df = pd.DataFrame({
    'Predicted_Points': puntos_predictions,
    'Predicted_Price': precio_predictions
}, index=X_test.index)

# Unir las predicciones con el DataFrame original usando el índice
df_with_predictions = df.join(predictions_df, how='inner')

# Ordenar el DataFrame de forma descendente por precio predicho y puntos predichos
df_sorted = df_with_predictions.sort_values(by=['Predicted_Price', 'Predicted_Points'], ascending=False)

# Impresión de la lista de jugadores ordenados por mayor precio y puntos predichos
print("Lista de jugadores ordenados por mayor precio y puntos predichos:")
print(df_sorted[['Nombre', 'Predicted_Price', 'Predicted_Points']])


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Lista de jugadores ordenados por mayor precio y puntos predichos:
            Nombre  Predicted_Price  Predicted_Points
0    J. Bellingham      4587.583008        196.664413
2     A. Griezmann      3707.730957        152.355103
10            Isco      3081.195312        131.446121
11           Sávio      2836.503906        124.952583
9       A. Budimir      2771.708496        132.352646
..             ...              ...               ...
317    L. Henrique       198.763000         27.601141
304    M. Morlanes       197.813110         27.729832
253        I. Baba       194.889969         35.532867
323       J. Navas       190.942734         23.828720
287          Chumi       189.083420         28.177731

[103 rows x 3 columns]


In [88]:
# Calcular el Error Cuadrático Medio (MSE) para puntos y precio
mse_puntos = mean_squared_error(y_puntos_test, puntos_predictions)
mse_precio = mean_squared_error(y_precio_test, precio_predictions)

# Calcular el coeficiente de determinación (R^2) para puntos y precio
r2_puntos = r2_score(y_puntos_test, puntos_predictions)
r2_precio = r2_score(y_precio_test, precio_predictions)

print("Calculo del error cuadratico y coeficiente de determinacion por puntos:")
print(f"MSE para puntos: {mse_puntos:.2f}")
print(f"R^2 para puntos: {r2_puntos:.2f}")

print("\n"+"Calculo del error cuadratico y coeficiente de determinacion por precios")
print(f"MSE para precio: {mse_precio:.2f}")
print(f"R^2 para precio: {r2_precio:.2f}")


Calculo del error cuadratico y coeficiente de determinacion por puntos:
MSE para puntos: 84.60
R^2 para puntos: 0.93

Calculo del error cuadratico y coeficiente de determinacion por precios
MSE para precio: 58076039249891.39
R^2 para precio: -0.59


In [89]:
# Crear DataFrames para los top 10 jugadores por precio y puntos predichos
top_10_precio = df_sorted.nlargest(10, 'Predicted_Price')
top_10_puntos = df_sorted.nlargest(10, 'Predicted_Points')

# Filtrar jugadores con la posición de delantero
delanteros = df_with_predictions[df_with_predictions['posicion'] == 'delantero']
mediocentro = df_with_predictions[df_with_predictions['posicion'] == 'mediocentro']
portero = df_with_predictions[df_with_predictions['posicion'] == 'portero']
defensa = df_with_predictions[df_with_predictions['posicion'] == 'defensa']

# Crear DataFrames para los top 10 Delanteros por precio y puntos predichos
top_10_precio_Delanteros = delanteros.nlargest(10, 'Predicted_Price')
top_10_puntos_Delanteros = delanteros.nlargest(10, 'Predicted_Points')

# Crear DataFrames para los top 10 Mediocentros por precio y puntos predichos
top_10_precio_Mediocentro = mediocentro.nlargest(10, 'Predicted_Price')
top_10_puntos_Mediocentro = mediocentro.nlargest(10, 'Predicted_Points')

# Crear DataFrames para los top 10 Porteros por precio y puntos predichos
top_10_precio_Portero = portero.nlargest(10, 'Predicted_Price')
top_10_puntos_Portero = portero.nlargest(10, 'Predicted_Points')

# Crear DataFrames para los top 10 Defensas por precio y puntos predichos
top_10_precio_Defensa = defensa.nlargest(10, 'Predicted_Price')
top_10_puntos_Defensa = defensa.nlargest(10, 'Predicted_Points')

In [90]:
# Guardar los DataFrames en archivos CSV
top_10_precio.to_csv('top_10_precio_predictions.csv', index=False)
top_10_puntos.to_csv('top_10_puntos_predictions.csv', index=False)

top_10_precio_Delanteros.to_csv('top_10_precio_delanteros_predictions.csv', index=False)
top_10_puntos_Delanteros.to_csv('top_10_puntos_delanteros_predictions.csv', index=False)

top_10_precio_Mediocentro.to_csv('top_10_precio_mediocentros_predictions.csv', index=False)
top_10_puntos_Mediocentro.to_csv('top_10_puntos_mediocentros_predictions.csv', index=False)

top_10_precio_Portero.to_csv('top_10_precio_porteros_predictions.csv', index=False)
top_10_puntos_Portero.to_csv('top_10_puntos_porteros_predictions.csv', index=False)

top_10_precio_Defensa.to_csv('top_10_precio_defensas_predictions.csv', index=False)
top_10_puntos_Defensa.to_csv('top_10_puntos_defensa_predictions.csv', index=False)

In [91]:
# Mostrar los DataFrames creados
print("Top 10 jugadores por precio predicho:")
print(top_10_precio[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("Top 10 jugadores por puntos predichos:")
print(top_10_puntos[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("\nTop 10 delanteros por precio predicho:")
print(top_10_precio_Delanteros[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("Top 10 delanteros por puntos predichos:")
print(top_10_puntos_Delanteros[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("\nTop 10 mediocentros por precio predicho:")
print(top_10_precio_Mediocentro[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("Top 10 mediocentros por puntos predichos:")
print(top_10_puntos_Mediocentro[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("\nTop 10 porteros por precio predicho:")
print(top_10_precio_Portero[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("Top 10 porteros por puntos predichos:")
print(top_10_puntos_Portero[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("\nTop 10 defensas por precio predicho:")
print(top_10_precio_Defensa[['Nombre', 'Predicted_Price', 'Predicted_Points']])

print("Top 10 defensas por puntos predichos:")
print(top_10_puntos_Defensa[['Nombre', 'Predicted_Price', 'Predicted_Points']])

Top 10 jugadores por precio predicho:
            Nombre  Predicted_Price  Predicted_Points
0    J. Bellingham      4587.583008        196.664413
2     A. Griezmann      3707.730957        152.355103
10            Isco      3081.195312        131.446121
11           Sávio      2836.503906        124.952583
9       A. Budimir      2771.708496        132.352646
22       G. Moreno      2660.482666        109.382050
18     F. Valverde      2633.853760        107.580612
101     F. De Jong      2355.942383         89.324203
30       B. Méndez      2198.858154         96.747337
72       V. Muriqi      2115.429199         92.848213
Top 10 jugadores por puntos predichos:
           Nombre  Predicted_Price  Predicted_Points
0   J. Bellingham      4587.583008        196.664413
2    A. Griezmann      3707.730957        152.355103
9      A. Budimir      2771.708496        132.352646
10           Isco      3081.195312        131.446121
11          Sávio      2836.503906        124.952583
22      G. 

In [95]:
top_10_precio

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Predicted_Points,Predicted_Price
0,J. Bellingham,24262000,10.2,173,mediocentro,Real Madrid Club de Fútbol,196.664413,4587.583008
2,A. Griezmann,24384000,7.6,144,delantero,Club Atlético de Madrid,152.355103,3707.730957
10,Isco,19605000,7.0,126,mediocentro,Real Betis Balompié,131.446121,3081.195312
11,Sávio,18030000,6.5,123,delantero,Girona Fútbol Club,124.952583,2836.503906
9,A. Budimir,15660000,6.7,127,delantero,Club Atlético Osasuna,132.352646,2771.708496
22,G. Moreno,18341000,6.5,104,delantero,Villarreal Club de Fútbol,109.382050,2660.482666
18,F. Valverde,20115000,5.7,108,mediocentro,Real Madrid Club de Fútbol,107.580612,2633.853760
101,F. De Jong,17706000,6.7,74,mediocentro,Fútbol Club Barcelona,89.324203,2355.942383
30,B. Méndez,15078000,5.7,97,mediocentro,Real Sociedad de Fútbol,96.747337,2198.858154
72,V. Muriqi,12454000,6.8,81,delantero,Real Club Deportivo Mallorca,92.848213,2115.429199


In [97]:
top_10_precio_Delanteros

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Predicted_Points,Predicted_Price
2,A. Griezmann,24384000,7.6,144,delantero,Club Atlético de Madrid,152.355103,3707.730957
11,Sávio,18030000,6.5,123,delantero,Girona Fútbol Club,124.952583,2836.503906
9,A. Budimir,15660000,6.7,127,delantero,Club Atlético Osasuna,132.352646,2771.708496
22,G. Moreno,18341000,6.5,104,delantero,Villarreal Club de Fútbol,109.382050,2660.482666
72,V. Muriqi,12454000,6.8,81,delantero,Real Club Deportivo Mallorca,92.848213,2115.429199
399,M. Guiu,204000,11.0,11,delantero,Fútbol Club Barcelona,80.057602,1610.869141
73,F. Torres,8080000,4.5,81,delantero,Fútbol Club Barcelona,78.985207,1315.419556
82,S. Omorodion,5889000,4.4,79,delantero,Deportivo Alavés,79.151733,1132.272949
104,A. Prats,2871000,4.6,74,delantero,Real Club Deportivo Mallorca,80.063919,941.878662
84,L. Ramazani,3171000,4.2,79,delantero,Unión Deportiva Almería,81.072693,919.127991


In [98]:
top_10_precio_Mediocentro

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Predicted_Points,Predicted_Price
0,J. Bellingham,24262000,10.2,173,mediocentro,Real Madrid Club de Fútbol,196.664413,4587.583008
10,Isco,19605000,7.0,126,mediocentro,Real Betis Balompié,131.446121,3081.195312
18,F. Valverde,20115000,5.7,108,mediocentro,Real Madrid Club de Fútbol,107.580612,2633.853760
101,F. De Jong,17706000,6.7,74,mediocentro,Fútbol Club Barcelona,89.324203,2355.942383
30,B. Méndez,15078000,5.7,97,mediocentro,Real Sociedad de Fútbol,96.747337,2198.858154
68,V. Tsygankov,14504000,5.9,82,mediocentro,Girona Fútbol Club,85.511063,2052.852539
55,Y. Herrera,11017000,5.7,86,mediocentro,Girona Fútbol Club,90.100403,1826.176270
39,I. Palazón,12991000,4.8,92,mediocentro,Rayo Vallecano de Madrid,85.826653,1810.294067
33,S. Arribas,11043000,5.1,96,mediocentro,Unión Deportiva Almería,94.722252,1795.082764
93,I. Rakitić,14477000,4.5,76,mediocentro,Sevilla Fútbol Club,70.440521,1672.296387


In [99]:
top_10_precio_Portero

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Predicted_Points,Predicted_Price
222,K. Arrizabalaga,7230000,4.4,48,portero,Real Madrid Club de Fútbol,46.965225,902.669800
237,A. Lunin,3059000,5.6,45,portero,Real Madrid Club de Fútbol,59.965176,900.439819
227,R. Silva,5232000,4.7,47,portero,Real Betis Balompié,51.277008,845.240479
324,I. Peña,3576000,4.8,29,portero,Fútbol Club Barcelona,35.889339,625.821594
195,A. Ferreira,1918000,3.5,52,portero,Granada Club de Fútbol,50.115906,483.283539
504,I. Grbić,160000,0.0,0,portero,Club Atlético de Madrid,4.744431,334.773315
505,J. Doménech,160000,0.0,0,portero,Valencia Club de Fútbol,4.744431,334.773315
509,P. Reina,160000,0.0,0,portero,Villarreal Club de Fútbol,4.744431,334.773315
499,A. López,164000,0.0,0,portero,Granada Club de Fútbol,4.736802,334.712158
484,J. Agirrezabala,208000,0.0,0,portero,Athletic Club,4.652873,334.039551


In [100]:
top_10_precio_Defensa

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Predicted_Points,Predicted_Price
76,J. Cancelo,16261000,5.0,80,defensa,Fútbol Club Barcelona,78.145569,1940.641235
63,A. Witsel,10736000,4.7,85,defensa,Club Atlético de Madrid,80.978134,1571.514526
79,S. Cardona,11393000,4.4,80,defensa,Unión Deportiva Las Palmas,72.760590,1493.991333
131,C. Mosquera,6802000,3.9,67,defensa,Valencia Club de Fútbol,61.899185,953.646545
176,A. Balde,6306000,3.3,56,defensa,Fútbol Club Barcelona,46.611984,690.679626
167,A. Suárez,2489000,3.8,57,defensa,Unión Deportiva Las Palmas,57.079536,611.413269
209,M. Diakhaby,1456000,4.2,50,defensa,Valencia Club de Fútbol,54.984299,580.977173
203,M. Nastasić,2359000,3.6,51,defensa,Real Club Deportivo Mallorca,49.568596,515.894348
194,A. Catena,2473000,3.5,52,defensa,Club Atlético Osasuna,49.517265,510.234650
275,L. Hernández,440000,4.6,37,defensa,Cádiz Club de Fútbol,46.692444,501.255554


In [124]:
####